In [ ]:
import sys
if 'google.colab' in sys.modules:
    # Installing packages in Google Colab environment
    !pip install datasets transformers

    # Mounting google drive to enable access to data files
    from google.colab import drive
    drive.mount('/content/drive')

    # Changing working directory to personality
    %cd /content/drive/MyDrive/LLM4BeSci/personality

## Processing data

In [ ]:
import pandas as pd
from datasets import Dataset
from transformers import AutoTokenizer

In [ ]:
# Loading data with pandas
neo_items =  pd.read_csv('NEO_items.csv', usecols=['item', 'construct'])
neo_items

In [ ]:
# Converting into a HuggingFace dataset
dat = Dataset.from_pandas(neo_items)
dat

In [ ]:
# Loading the tokenizer
model_ckpt = 'distilbert-base-uncased'
tokenizer = AutoTokenizer.from_pretrained(model_ckpt)
print(f'Vocabulary size: {tokenizer.vocab_size}, max context length: {tokenizer.model_max_length}')

In [ ]:
# Tokenizing the text
batch_tokenizer = lambda x: tokenizer(x['item'], padding=True, truncation=True)
dat = dat.map(batch_tokenizer, batched=True, batch_size=None)
print([tokenizer.decode(id) for id in dat['input_ids'][0]])

In [ ]:
# Setting the format of the dataset to torch tensors for passing to the model
dat.set_format('torch', columns=['input_ids', 'attention_mask'])
dat

In [ ]:
dat['input_ids'].shape

# Feature extraction

In [ ]:
import torch
from transformers import AutoModel

In [ ]:
# Loading the model and moving it to the GPU if available
if torch.cuda.is_available():  # for nvidia GPUs
    device = torch.device('cuda')
elif torch.backends.mps.is_available(): # for Apple Metal Performance Sharder (mps) GPUs
    device = torch.device('mps')
else:
    device = torch.device('cpu')

device

In [ ]:
# Loading the model
model = AutoModel.from_pretrained('distilbert-base-uncased').to(device)
f'Model inputs: {tokenizer.model_input_names}'

In [ ]:
def extract_features(batch):
    """Extract features from a batch of items"""
    inputs = {k:v.to(device) for k, v in batch.items() if k in tokenizer.model_input_names}
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
        return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}


dat = dat.map(extract_features, batched=True, batch_size=8)
dat

In [ ]:
dat['input_ids'].shape

# Comparing predicted and observed construct similarities

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [ ]:
# Converting the hidden state into a data frame for easy manipulation
features = pd.DataFrame(dat['hidden_state'])
features

In [ ]:
# Calculating the cosine similarity between construct embeddings
sims = pd.DataFrame(
    cosine_similarity(features),
    index=neo_items['item'],
    columns=neo_items['item'],
)
sims

In [ ]:
# Loading observed correlations and pivoting to a correlation matrix
sims_observed = pd.read_csv('item_corrs.csv')
sims_observed

In [ ]:
# Pivoting to a correlation matrix for easy comparison with predicted correlations
sims_observed = sims_observed.pivot(index='text_i', columns='text_j', values='cor')
sims_observed

In [ ]:
# Aligning rows and columns the predicted and observed correlations
sims, sims_observed = sims.align(sims_observed)


def lower_triangle_flat(df):
    """Takes the lower triangle of a dataframe and flattens it into a vector"""
    rows, cols = np.triu_indices(len(df), k=1)  # k=1 to exclude the diagonal (self-similarities)
    return pd.Series(df.values[rows, cols])


sims, sims_observed = lower_triangle_flat(sims), lower_triangle_flat(sims_observed)

# Correlation between predicted and observed
print(f'r: {sims.corr(sims_observed).round(2)}')
print(f'r of absolute values: {sims.abs().corr(sims_observed.abs()).round(2)}')